In [1]:
import sys, os
sys.path.append('/Users/benjamin/Desktop/repos/chi-data/backend/aws') 

from s3 import S3
from dynamo import DynamoConn
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
chicago_tracts = ['17031839000', '17031842400', '17031840300', '17031841100', '17031841200', '17031030601', '17031838200', '17031650301', '17031530503', '17031760803', '17031030604', '17031020801', '17031080202', '17031070102', '17031540102', '17031843300', '17031540101', '17031440201', '17031031501', '17031031502', '17031834900', '17031834800', '17031160502', '17031140702', '17031842000', '17031150402', '17031834400', '17031040201', '17031040202', '17031020702', '17031020802', '17031020301', '17031838700', '17031841500', '17031063301', '17031838600', '17031837300', '17031837800', '17031030102', '17031837400', '17031836900', '17031020601', '17031020602', '17031020302', '17031020902', '17031010701', '17031030103', '17031030104', '17031010502', '17031010503', '17031010702', '17031837100', '17031063303', '17031837000', '17031010201', '17031020901', '17031040401', '17031210602', '17031160501', '17031191302', '17031190602', '17031840200', '17031530501', '17031020701', '17031831400', '17031831300', '17031010501', '17031671200', '17031833000', '17031760801', '17031252101', '17031252102', '17031252202', '17031220601', '17031210502', '17031220602', '17031220901', '17031220902', '17031200401', '17031200402', '17031191301', '17031190601', '17031190701', '17031190702', '17031190401', '17031190402', '17031832100', '17031843800', '17031842800', '17031030101', '17031980100', '17031833300', '17031832900', '17031838000', '17031838100', '17031030706', '17031833100', '17031839100', '17031081201', '17031150401', '17031150501', '17031843000', '17031842900', '17031842100', '17031841900', '17031080201', '17031070101', '17031081202', '17031040402', '17031220702', '17031140301', '17031842200', '17031843700', '17031835000', '17031842300', '17031390400', '17031835100', '17031834700', '17031151001', '17031140602', '17031151002', '17031160602', '17031140601', '17031140701', '17031841800', '17031842500', '17031842600', '17031843500', '17031843400', '17031140302', '17031110501', '17031351500', '17031843200', '17031840800', '17031110502', '17031760802', '17031150502', '17031843100', '17031834300', '17031843900', '17031381500', '17031831200', '17031831600', '17031831500', '17031580800', '17031831100', '17031831700', '17031271500', '17031440202', '17031440102', '17031430102', '17031030702', '17031030603', '17031030701', '17031836300', '17031838300', '17031839800', '17031836600', '17031730202', '17031430101', '17031063302', '17031836200', '17031300900', '17031840100', '17031580502', '17031841400', '17031081401', '17031081403', '17031030703', '17031836000', '17031835800', '17031835900', '17031834500', '17031843600', '17031310400', '17031834600', '17031831800', '17031832600', '17031061901', '17031839900', '17031071700', '17031770602', '17031839700', '17031835200', '17031833900', '17031836800', '17031838800', '17031836700', '17031832200', '17031252201', '17031831000', '17031832400', '17031070103', '17031220701', '17031530100', '17031770902', '17031660301', '17031660302', '17031081100', '17031834000', '17031834200', '17031301802', '17031835500', '17031839500', '17031839200', '17031841600', '17031840400', '17031841000', '17031730201', '17031580501', '17031700302', '17031700402', '17031700401', '17031840700', '17031222900', '17031830500', '17031841700', '17031071600', '17031841300', '17031301801', '17031530502', '17031071400', '17031832300', '17031832500', '17031832000', '17031160601', '17031210501', '17031210601', '17031650302', '17031700502', '17031836100', '17031835700', '17031836400', '17031836500', '17031839600', '17031840000', '17031835600', '17031830800', '17031460301', '17031301701', '17031150700', '17031640100', '17031081300', '17031301702', '17031230100', '17031700301', '17031490902', '17031490901', '17031831900', '17031830900', '17031460302', '17031830700', '17031830600', '17031700501', '17031431302', '17031431301', '17031440101', '17031301803', '17031310200', '17031310300', '17031310500', '17031140500', '17031510300', '17031491400', '17031140800', '17031500200', '17031530200', '17031740100', '17031150200', '17031150300', '17031301100', '17031301200', '17031740400', '17031640300', '17031640600', '17031010400', '17031071800', '17031020400', '17031160300', '17031310700', '17031071500', '17031670500', '17031670600', '17031670700', '17031670800', '17031670900', '17031671100', '17031671300', '17031671400', '17031671500', '17031671600', '17031670100', '17031230500', '17031230600', '17031240600', '17031241000', '17031191000', '17031080300', '17031300800', '17031650200', '17031660500', '17031221400', '17031161000', '17031340600', '17031240500', '17031240700', '17031240800', '17031240900', '17031241100', '17031190300', '17031190800', '17031190900', '17031191100', '17031191200', '17031080100', '17031080400', '17031081000', '17031150600', '17031301600', '17031020500', '17031821402', '17031750300', '17031640800', '17031750100', '17031740200', '17031650400', '17031650500', '17031660600', '17031221100', '17031221300', '17031221500', '17031020100', '17031221600', '17031160800', '17031160900', '17031222800', '17031161100', '17031161200', '17031161300', '17031650100', '17031330200', '17031420600', '17031420700', '17031661000', '17031823304', '17031010100', '17031230300', '17031640400', '17031640500', '17031640700', '17031151200', '17031310800', '17031660400', '17031670400', '17031340400', '17031420800', '17031421200', '17031661100', '17031222500', '17031190200', '17031390600', '17031010300', '17031010600', '17031020200', '17031150800', '17031151100', '17031230200', '17031160100', '17031160200', '17031160400', '17031310600', '17031310900', '17031320400', '17031390500', '17031320600', '17031740300', '17031660700', '17031222700', '17031221200', '17031160700', '17031222200', '17031340500', '17031330100', '17031340300', '17031350400', '17031750500', '17031750400', '17031750200', '17031430200', '17031660800', '17031660900', '17031670200', '17031670300', '17031222600', '17031230400', '17031170400', '17031380700', '17031381900', '17031390200', '17031171000', '17031180100', '17031390300', '17031390700', '17031400300', '17031400400', '17031400500', '17031400800', '17031410100', '17031440300', '17031440700', '17031430800', '17031671800', '17031680500', '17031681100', '17031200100', '17031410200', '17031170600', '17031380500', '17031381700', '17031430300', '17031430700', '17031680600', '17031430900', '17031431200', '17031431400', '17031230800', '17031231100', '17031170900', '17031210100', '17031420200', '17031420300', '17031440600', '17031440800', '17031440900', '17031450300', '17031671900', '17031672000', '17031680900', '17031681000', '17031240200', '17031240300', '17031200200', '17031410500', '17031410600', '17031170100', '17031170200', '17031170300', '17031381800', '17031170500', '17031351400', '17031381200', '17031381400', '17031390100', '17031430400', '17031430500', '17031430600', '17031230700', '17031230900', '17031231200', '17031170700', '17031170800', '17031171100', '17031190100', '17031242200', '17031200300', '17031210400', '17031220300', '17031210700', '17031210800', '17031210900', '17031420100', '17031420400', '17031420500', '17031691300', '17031560400', '17031470100', '17031480100', '17031691000', '17031242300', '17031411100', '17031460400', '17031681200', '17031690500', '17031241400', '17031241500', '17031241600', '17031242000', '17031243300', '17031491200', '17031480300', '17031710800', '17031711300', '17031010202', '17031480200', '17031480400', '17031480500', '17031241200', '17031691100', '17031691200', '17031691400', '17031031700', '17031691500', '17031700100', '17031410700', '17031410800', '17031410900', '17031411000', '17031411200', '17031530600', '17031460100', '17031460200', '17031460500', '17031460600', '17031460700', '17031241300', '17031461000', '17031681300', '17031681400', '17031050700', '17031690300', '17031690400', '17031690900', '17031050900', '17031242100', '17031243100', '17031243200', '17031051100', '17031243400', '17031243500', '17031711100', '17031250200', '17031491000', '17031491100', '17031510100', '17031510200', '17031710100', '17031520200', '17031710900', '17031711000', '17031711200', '17031711400', '17031081402', '17031710300', '17031061902', '17031030200', '17031030300', '17031030400', '17031030500', '17031030800', '17031242400', '17031242500', '17031242600', '17031242700', '17031242800', '17031242900', '17031243000', '17031220400', '17031220500', '17031710600', '17031221000', '17031810400', '17031031800', '17031520500', '17031720500', '17031490300', '17031490400', '17031350100', '17031351100', '17031251200', '17031570100', '17031030900', '17031250600', '17031560100', '17031031900', '17031490500', '17031490600', '17031490700', '17031490800', '17031700200', '17031710200', '17031710400', '17031710500', '17031710700', '17031032100', '17031251000', '17031280400', '17031500300', '17031320100', '17031351000', '17031360200', '17031380100', '17031231500', '17031251100', '17031560800', '17031550200', '17031560900', '17031561000', '17031561100', '17031570200', '17031730400', '17031730500', '17031730600', '17031730700', '17031250300', '17031250400', '17031250500', '17031250700', '17031250800', '17031520300', '17031520600', '17031560200', '17031560300', '17031560700', '17031711500', '17031720600', '17031031000', '17031580300', '17031050500', '17031050800', '17031051000', '17031051400', '17031081900', '17031252000', '17031260300', '17031040300', '17031040800', '17031050100', '17031050200', '17031050300', '17031380200', '17031051200', '17031081500', '17031720100', '17031720200', '17031720300', '17031720700', '17031730100', '17031060100', '17031730300', '17031031100', '17031031200', '17031031300', '17031060200', '17031031400', '17031570500', '17031580100', '17031611600', '17031580200', '17031580400', '17031580600', '17031051300', '17031530300', '17031500100', '17031050600', '17031081800', '17031090100', '17031090200', '17031090300', '17031100100', '17031260200', '17031100200', '17031260100', '17031062400', '17031491300', '17031720400', '17031530400', '17031063400', '17031750600', '17031040100', '17031040600', '17031040700', '17031040900', '17031980000', '17031081600', '17031081700', '17031251300', '17031251400', '17031251500', '17031251900', '17031260700', '17031600900', '17031611200', '17031062800', '17031062900', '17031063000', '17031063100', '17031063200', '17031600400', '17031060300', '17031060800', '17031061100', '17031061800', '17031062200', '17031110200', '17031251600', '17031251700', '17031251800', '17031570300', '17031570400', '17031100600', '17031062600', '17031100700', '17031261000', '17031062700', '17031110100', '17031260900', '17031270500', '17031520100', '17031611400', '17031610300', '17031610800', '17031611000', '17031120300', '17031271800', '17031520400', '17031061200', '17031260500', '17031260600', '17031580700', '17031061500', '17031590500', '17031590600', '17031590700', '17031600600', '17031600700', '17031060400', '17031060500', '17031060900', '17031061000', '17031062000', '17031062100', '17031260400', '17031260800', '17031071000', '17031100300', '17031100400', '17031071200', '17031100500', '17031283100', '17031271200', '17031610400', '17031611100', '17031611300', '17031062300', '17031062500', '17031070700', '17031071100', '17031071300', '17031140200', '17031120400', '17031130100', '17031130200', '17031140100', '17031283200', '17031140400', '17031292400', '17031282700', '17031292500', '17031300600', '17031300700', '17031282800', '17031283800', '17031290900', '17031291600', '17031630200', '17031630800', '17031271300', '17031280900', '17031611700', '17031612100', '17031070300', '17031070600', '17031300500', '17031291200', '17031612000', '17031620300', '17031620400', '17031630100', '17031070500', '17031630300', '17031630400', '17031630500', '17031110300', '17031110400', '17031120200', '17031120100', '17031271400', '17031280100', '17031280800', '17031281900', '17031611900', '17031550100', '17031611500', '17031611800', '17031620100', '17031620200', '17031070200', '17031070400', '17031130300', '17031292200', '17031630900']

In [2]:
import s3fs
s3 = s3fs.S3FileSystem()
dataset = 'lehd_od'
year = '2002'
# select only JT00
ds = pq.ParquetDataset(path_or_paths='bnroths/chicago-data/lehd_od/year=2002/il_lehd_od_main_JT00_2002.parquet', filesystem=s3, validate_schema=False)

In [3]:
table = ds.read()
df = table.to_pandas()

In [4]:
df.head()

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate,h_tract,w_tract
0,170010001001001,170010104003020,1,0,1,0,0,0,1,0,0,1,20160228,17001010400,17001000100
1,170010001001003,170010011001025,1,0,1,0,0,0,1,0,1,0,20160228,17001001100,17001000100
2,170010001001003,170010011002036,1,0,1,0,0,0,1,0,1,0,20160228,17001001100,17001000100
3,170010001001003,170010101004121,1,1,0,0,0,1,0,0,1,0,20160228,17001010100,17001000100
4,170010001001003,170010106003018,1,0,0,1,0,0,1,0,1,0,20160228,17001010600,17001000100


In [68]:
df.head()
df = df[df['h_tract'] != '17031671800']#.isin(chicago_tracts)]
df = df[df['w_tract'].isin(chicago_tracts)]

In [64]:
df = df[df['w_tract'].isin(chicago_tracts)]

In [69]:
print (set(df.h_tract) - set(df.w_tract))
print (set(df.w_tract) - set(df.h_tract))

set([])
set([])


In [70]:
# select total jobs data
pivot = pd.pivot_table(df, values='S000', columns=['w_tract'], index=['h_tract'], aggfunc=np.sum)

In [71]:
pivot = pivot.fillna(0)
pivot.head()

w_tract,17031010100,17031010201,17031010202,17031010300,17031010400,17031010501,17031010502,17031010503,17031010600,17031010701,...,17031843200,17031843300,17031843400,17031843500,17031843600,17031843700,17031843800,17031843900,17031980000,17031980100
h_tract,,,,,,,,,,,,,,,,,,,,,
17031010100,46.0,5.0,7.0,20.0,10.0,0.0,0.0,0.0,4.0,0.0,...,5.0,0.0,0.0,1.0,0.0,11.0,0.0,0.0,20.0,1.0
17031010201,14.0,13.0,22.0,11.0,7.0,1.0,4.0,1.0,23.0,0.0,...,6.0,1.0,0.0,1.0,0.0,5.0,0.0,0.0,18.0,1.0
17031010202,5.0,9.0,9.0,2.0,1.0,0.0,4.0,0.0,4.0,0.0,...,2.0,1.0,0.0,3.0,0.0,3.0,0.0,0.0,5.0,0.0
17031010300,16.0,5.0,10.0,54.0,28.0,1.0,0.0,2.0,23.0,0.0,...,4.0,2.0,1.0,2.0,0.0,9.0,0.0,0.0,22.0,1.0
17031010400,5.0,2.0,1.0,7.0,196.0,0.0,0.0,0.0,5.0,0.0,...,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,15.0,0.0


In [72]:
# column totals (w_tract)
w_tracts = pivot.sum()
# row totals (h_tracts)
h_tracts = pivot.transpose().sum()

In [73]:
# check work, transpose commuting matrix and divide by sum, should output w_tracts
np.matmul((pivot.transpose()/h_tracts), h_tracts)

array([  3.24000000e+02,   1.93000000e+02,   2.70000000e+02,
         5.85000000e+02,   2.57300000e+03,   3.40000000e+01,
         7.50000000e+01,   1.33000000e+02,   5.40000000e+02,
         2.10000000e+01,   4.36000000e+02,   3.60000000e+02,
         4.17000000e+02,   5.02000000e+02,   5.41000000e+02,
         5.15000000e+02,   4.15000000e+02,   2.67000000e+02,
         4.72000000e+02,   2.87000000e+02,   3.17000000e+02,
         5.53000000e+02,   6.96000000e+02,   8.50000000e+01,
         6.80000000e+01,   1.70000000e+02,   1.23000000e+02,
         1.60000000e+02,   1.73000000e+02,   5.01000000e+02,
         1.40000000e+02,   3.31000000e+02,   4.60000000e+02,
         1.29000000e+02,   7.60000000e+01,   1.78100000e+03,
         1.35000000e+02,   1.09000000e+03,   2.21000000e+02,
         8.72000000e+02,   7.86000000e+02,   7.76000000e+02,
         1.15900000e+03,   7.42000000e+02,   8.12000000e+02,
         6.58000000e+02,   1.36700000e+03,   2.00000000e+02,
         3.30000000e+02,

In [74]:
print w_tracts

w_tract
17031010100      324.0
17031010201      193.0
17031010202      270.0
17031010300      585.0
17031010400     2573.0
17031010501       34.0
17031010502       75.0
17031010503      133.0
17031010600      540.0
17031010701       21.0
17031010702      436.0
17031020100      360.0
17031020200      417.0
17031020301      502.0
17031020302      541.0
17031020400      515.0
17031020500      415.0
17031020601      267.0
17031020602      472.0
17031020701      287.0
17031020702      317.0
17031020801      553.0
17031020802      696.0
17031020901       85.0
17031020902       68.0
17031030101      170.0
17031030102      123.0
17031030103      160.0
17031030104      173.0
17031030200      501.0
                ...   
17031841100     1945.0
17031841200      542.0
17031841300     1342.0
17031841400       45.0
17031841500       60.0
17031841600       24.0
17031841700      222.0
17031841800      351.0
17031841900     9573.0
17031842000     6568.0
17031842100     1251.0
17031842200     3723.0
170

In [75]:
A = pivot.transpose()/h_tracts

In [76]:
A.head()

h_tract,17031010100,17031010201,17031010202,17031010300,17031010400,17031010501,17031010502,17031010503,17031010600,17031010701,...,17031843200,17031843300,17031843400,17031843500,17031843600,17031843700,17031843800,17031843900,17031980000,17031980100
w_tract,,,,,,,,,,,,,,,,,,,,,
17031010100,0.041971,0.011354,0.009747,0.010343,0.003482,0.004136,0.005230,0.000000,0.002711,0.001198,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000859,0.0,0.0
17031010201,0.004562,0.010543,0.017544,0.003232,0.001393,0.004136,0.003138,0.000000,0.001085,0.002395,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000859,0.0,0.0
17031010202,0.006387,0.017843,0.017544,0.006464,0.000696,0.000827,0.003138,0.001610,0.002711,0.002395,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000859,0.0,0.0
17031010300,0.018248,0.008921,0.003899,0.034906,0.004875,0.004963,0.009414,0.000000,0.009219,0.007186,...,0.0,0.000000,0.000000,0.000000,0.001264,0.0,0.000000,0.000000,0.0,0.0
17031010400,0.009124,0.005677,0.001949,0.018100,0.136490,0.043011,0.031381,0.043478,0.044469,0.008383,...,0.0,0.002625,0.005376,0.006383,0.002528,0.0,0.001597,0.001718,0.0,0.0


In [103]:
w, v = np.linalg.eig(A)

In [104]:
pivot.shape

(800, 800)

In [105]:
print type(w[0])
print sum(v[:, 0])
from numpy import linalg as LA
print LA.norm(v[:, 0])
print v[:, 0]
#sum(x**2 for x in v[0])
# print w[0].real, w[0].imag
# print w[0]
# 
# print w[1].real, w[1].imag
# print w[1]
# 
# print w[3].real, w[3].imag
# print round(w[3], 4)
# print str(w[3])


<type 'numpy.complex128'>
(-3.37814333252+0j)
1.0
[ -3.90693476e-04+0.j  -2.71567967e-04+0.j  -3.18383553e-04+0.j
  -6.94849565e-04+0.j  -9.45949940e-03+0.j  -4.53820732e-05+0.j
  -8.99516071e-05+0.j  -1.86085077e-04+0.j  -7.06391078e-04+0.j
  -7.74774289e-05+0.j  -3.98942735e-04+0.j  -3.45540106e-04+0.j
  -5.56812193e-04+0.j  -5.52818907e-04+0.j  -9.80781979e-04+0.j
  -7.73521155e-04+0.j  -3.60287812e-04+0.j  -4.71079983e-04+0.j
  -1.62227681e-03+0.j  -4.66136904e-04+0.j  -3.86156572e-04+0.j
  -8.50909643e-04+0.j  -1.03652043e-03+0.j  -1.15847880e-04+0.j
  -8.33258491e-04+0.j  -3.91547766e-04+0.j  -3.11077702e-04+0.j
  -3.09599221e-04+0.j  -4.79011825e-04+0.j  -8.57080383e-04+0.j
  -1.69996245e-04+0.j  -4.70577239e-04+0.j  -5.17993238e-04+0.j
  -2.10214892e-04+0.j  -2.21687379e-04+0.j  -3.18832404e-03+0.j
  -1.12428405e-03+0.j  -5.83981643e-03+0.j  -1.52886748e-03+0.j
  -3.25051992e-03+0.j  -1.16812562e-03+0.j  -1.81846502e-03+0.j
  -1.91781307e-03+0.j  -1.24772906e-03+0.j  -1.2767122

  -7.15716017e-02+0.j  -6.96907452e-03+0.j]


In [80]:
idx = w.argsort()[::-1]
eigenValues = w[idx]
eigenVectors = v[:,idx]
# for i in range(10):
#     print type(eigenVectors[i]), type(eigenValues[i])
#     print eigenVectors[i]
#     for i in
    
for a in eigenVectors[0]:
    print round(1000* a.real, 1), a.imag
    
transformed = [round(1000*x.real, 1) for x in eigenVectors[0]]
has_imag    = [1 if x.imag != 0 else 0 for x in eigenVectors[0]]
print transformed[0:5]
print has_imag[0:5]


40.8 0.0
-20.8 0.0
27.7 0.0097157512353
27.7 -0.0097157512353
-11.6 0.0
10.6 0.00133167499606
10.6 -0.00133167499606
-8.1 0.0
52.3 0.0
40.5 0.0
22.2 0.0
14.8 0.00124438008998
14.8 -0.00124438008998
-45.0 0.003094476784
-45.0 -0.003094476784
-32.1 0.0296802880471
-32.1 -0.0296802880471
58.5 0.0
55.1 0.0
24.1 0.0234315590824
24.1 -0.0234315590824
-1.4 0.0107605551388
-1.4 -0.0107605551388
7.4 -0.0103485729909
7.4 0.0103485729909
-8.2 -0.00303327216189
-8.2 0.00303327216189
-34.2 0.0
-0.5 -0.00667955369275
-0.5 0.00667955369275
6.1 -0.0325268443948
6.1 0.0325268443948
-112.0 -0.0436982020666
-112.0 0.0436982020666
-56.3 -0.00405678785432
-56.3 0.00405678785432
-34.2 0.0850715347388
-34.2 -0.0850715347388
59.8 -0.00812947295827
59.8 0.00812947295827
-1.1 0.0
3.6 -0.058694010218
3.6 0.058694010218
53.1 0.0932053162021
53.1 -0.0932053162021
-94.3 0.0
76.4 0.0372021399512
76.4 -0.0372021399512
-1.8 0.0116105695631
-1.8 -0.0116105695631
-33.3 0.0253108384275
-33.3 -0.0253108384275
-11.0 -0.001